In [52]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample


def FrankeFunction(x, y):
    term1 = 0.75 * np.exp(-(0.25 * (9 * x - 2)**2) - 0.25 * ((9 * y - 2)**2))
    term2 = 0.75 * np.exp(-((9 * x + 1)**2) / 49.0 - 0.1 * (9 * y + 1))
    term3 = 0.5 * np.exp(-(9 * x - 7)**2 / 4.0 - 0.25 * ((9 * y - 3)**2))
    term4 = -0.2 * np.exp(-(9 * x - 4)**2 - (9 * y - 7)**2)
    return term1 + term2 + term3 + term4


def create_X(x, y, n):
    """Returns the design matrix X from coordinates x and y with n polynomial degrees."""
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n + 1) * (n + 2) / 2)  # Number of elements in beta
    X = np.ones((N, l))

    for i in range(1, n + 1):
        q = int(i * (i + 1) / 2)
        for k in range(i + 1):
            X[:, q + k] = (x**(i - k)) * (y**k)

    return X


def MSE_bootstrap(y_actual, y_model):
    """Returns the mean squared error of the two arrays."""
    return np.mean(np.mean((y_actual - y_model)**2, axis=1, keepdims=True))


def bias_bootstrap(y_actual, y_model):
    """Returns the bias of the two arrays."""
    return np.mean((y_actual - np.mean(y_model, axis=1, keepdims=True))**2)


def variance_bootstrap(y):
    """Returns the variance of the array"""
    return np.mean(np.var(y, axis=1, keepdims=True))

# GLOBAL PARAMETERS
N = 1000  # number of data points
n_bootstraps = 100  # no. bootstrap resamples
noise_std = 1  # standard deviation for noise

# Figure output directory
RESULTS_DIR = Path("../results").resolve()
FIGURES_DIR = RESULTS_DIR / "figures"

# Create them if they dont exist
if not RESULTS_DIR.exists():
    RESULTS_DIR.mkdir()

if not FIGURES_DIR.exists():
    FIGURES_DIR.mkdir()

# Create data set
np.random.seed(2023)
x = np.sort(np.random.uniform(0, 1, N))
y = np.sort(np.random.uniform(0, 1, N))

# Get franke function with noise
noise = np.random.normal(0, noise_std, x.shape)
z = FrankeFunction(x, y) + noise

# Initialize scaler
scaler = StandardScaler()

In [53]:
######### BIAS-VARIANCE AS FUNCTION OF POLY. DEGREE #############
# PARAMETERS
maxdegree = 6  # max polynomial degree for plotting

# Init arrays
degrees = np.arange(0, maxdegree, 1)
mse_arr = np.empty(degrees.shape)
bias_arr = np.empty_like(mse_arr)
var_arr = np.empty_like(mse_arr)

# Iterate through all the different polynomial degrees
for degree in degrees[1:]:
    # Create design matrix X
    X = create_X(x, y, n=degree)  # intercept included

    # Split in training and test data up to the given degree
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)
    
    # Scale data
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Bootstrap resampling
    z_predict = np.empty((z_test.shape[0], n_bootstraps))
    for i in range(n_bootstraps):
        X_, z_ = resample(X_train_scaled, z_train)
        
        # OLS regression
        beta = np.linalg.pinv(X_.T @ X_) @ X_.T @ z_
        z_predict[:, i] = X_test_scaled @ beta
        
    # Store the MSE, bias, and variance
    print(z_test.shape, z_predict.shape, z_predict.size)
    mse_arr[degree] = MSE_bootstrap(z_test, z_predict)
    bias_arr[degree] = bias_bootstrap(z_test, z_predict)
    var_arr[degree] = variance_bootstrap(z_predict)

(200,) (200, 100) 20000


ValueError: operands could not be broadcast together with shapes (200,) (200,100) 